In [1]:
# Install required libraries
!pip install chromadb --quiet
!pip install sentence-transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 5.4 MB/s eta 

In [2]:
# === IMPRTS ===
import os, time, torch, shutil, pickle
import chromadb
from chromadb import Client
from chromadb import Documents, EmbeddingFunction, Embeddings
from sentence_transformers import SentenceTransformer
from tqdm import trange

2025-07-26 10:12:45.449441: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753524765.790842      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753524765.891001      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
# === CONFIGURATION ===
NAME = "subsum_6"
START_ID = 3500000  # <-- set this manually per chunk

In [4]:
# === Load documents ===
with open('/kaggle/input/capstone-rag-data-toembed/submission_summary_list.pkl', 'rb') as f:
    doc_s = pickle.load(f)

doc_s = doc_s[3500000:4654210] # 1,000,000 to 1,499,999

In [5]:
# === Device config ===
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [6]:
# === Setup DB path ===
persist_dir = f"/kaggle/working/{NAME}_chroma_db"
os.makedirs(persist_dir, exist_ok=True)

In [7]:
# === Custom embedding function ===
class GPUSentenceTransformerEmbedding(EmbeddingFunction[Documents]):
    def __init__(self, model_name="BAAI/bge-base-en-v1.5"):
        print(f"Loading model: {model_name}")
        self.model = SentenceTransformer(model_name, device=device)
    
    def embed(self, input: Documents) -> Embeddings:
        start = time.time()
        embeddings = self.model.encode(
            input,
            device=device,
            batch_size=64,
            convert_to_tensor=False,
            show_progress_bar=False
        )
        elapsed = time.time() - start
        print(f"Encoded {len(input)} docs in {elapsed:.2f}s ({len(input)/elapsed:.0f} docs/sec)")
        return embeddings.tolist() if hasattr(embeddings, "tolist") else embeddings

gpu_embedding_function = GPUSentenceTransformerEmbedding()

Loading model: BAAI/bge-base-en-v1.5


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# === Setup Chroma client without embedding function ===
# We're storing embeddings directly, so embedding_function is not passed here
client = chromadb.PersistentClient(path=persist_dir)
collection = client.get_or_create_collection(name=NAME)

In [9]:
# === Batch embedding and storage ===
def embed_documents(docs, start_id=0, batch_size=100):
    total_docs = len(docs)
    for i in trange(0, total_docs, batch_size, desc="Embedding batches", unit="batch"):
        batch = docs[i:i+batch_size]
        texts = [doc["page_content"] for doc in batch]
        metadatas = [doc["metadata"] for doc in batch]
        ids = [f"{NAME}_{start_id + j}" for j in range(i, i + len(batch))]

        embeddings = gpu_embedding_function.embed(texts)

        collection.add(
            embeddings=embeddings,
            metadatas=metadatas,
            ids=ids
        )

        if torch.cuda.is_available() and i % 500 == 0:
            torch.cuda.empty_cache()


In [10]:
# === Run embedding ===
start_time = time.time()
embed_documents(doc_s, start_id=START_ID, batch_size=100)
# print(f"Completed embedding {len(doc_s)} docs in {time.time() - start_time:.1f} seconds")

Embedding batches:   0%|          | 1/11543 [00:01<5:54:34,  1.84s/batch]

Encoded 100 docs in 1.69s (59 docs/sec)


Embedding batches:   0%|          | 2/11543 [00:03<5:35:52,  1.75s/batch]

Encoded 100 docs in 1.58s (63 docs/sec)


Embedding batches:   0%|          | 3/11543 [00:04<5:06:42,  1.59s/batch]

...

Encoded 100 docs in 1.86s (54 docs/sec)


Embedding batches: 100%|█████████▉| 11542/11543 [5:53:59<00:02,  2.03s/batch]

Encoded 100 docs in 1.53s (65 docs/sec)


Embedding batches: 100%|██████████| 11543/11543 [5:53:59<00:00,  1.84s/batch]

Encoded 9 docs in 0.20s (44 docs/sec)


In [11]:
# === Archive DB ===
zip_path = f"/kaggle/working/{NAME}_chroma.zip"
shutil.make_archive(
    base_name=zip_path.replace(".zip", ""),
    format='zip',
    root_dir=persist_dir
)
print(f"Chroma DB zipped to: {zip_path}")

Chroma DB zipped to: /kaggle/working/subsum_6_chroma.zip
